To detect movements in a video, we use background subtraction. There are many different background subtraction algorithms, and each algorithm has various parameters. You can use the default parameter settings in Birdwatcher, but you can also modify the parameters.

This notebook introduces some of background subtractors that are implemented in Birdwatcher. How to access them, and look at the various parameters they have. And how they are used for movement detection. Much of what's in here can be encaspulated by higher-order functions or classes, but if you want to have full control over things, have a look at the following.

In [ ]:
import birdwatcher as bw

### Create a video object

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zf20s_low.mp4')

### Create a background subtractor object

Create a background subtractor object with default parameters. This object basically encapsulates the algorithm that determines the background from a history of images, and uses that to find what is not background in the current image. See opencv page for more info on the algorithm: https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html

In [ ]:
bgs = bw.BackgroundSubtractorMOG2()

What are the parameters?

In [ ]:
bgs.get_params()

In the docstrings, you can see the definition of each parameter:

In [ ]:
bw.BackgroundSubtractorMOG2?

You can use non-default parameters by specifying them at intstantiation:

In [ ]:
bgs = bw.BackgroundSubtractorMOG2(VarThreshold=50, NMixtures=8, History=4)

In [ ]:
bgs.get_params()

### Apply background subtractor to video Frames

This can be done by setting up a pipeline that generates and processes image sequences.

First, set up a frame generator that produces gray frames from the color video file object:

In [ ]:
frames = vfs.iter_frames(color=False)

Then, apply the background subtractor that we created above. It returns another frame generator.

In [ ]:
frames = frames.apply_backgroundsegmenter(bgs, learningRate=-1)

We now have a frame generator that produces foreground mask frames. Let's get rid of some noise. (Look at MorphologyEx page of opencv for what this does: https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html)

In [ ]:
frames = frames.morphologyex(morphtype='open', kernelsize=2)

We are ready to start running the whole frame pipeline and save results as a video for inspection.

In [ ]:
frames = frames.tovideo('output/test_MOG2.mp4', framerate=vfs.avgframerate)

This whole pipeline can also be shortened like so:

In [ ]:
frames = (vfs.iter_frames(color=False)
          .apply_backgroundsegmenter(bgs, learningRate=-1)
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test_MOG2.mp4', framerate=vfs.avgframerate))

### Another example

Here's another example of a pipeline. We have added a blur manipulation to the videoframes before appying the background segmenter. Also, we have added a region of interest (roi).

In [ ]:
frames = (vfs.iter_frames(color=False)
          .blur((10,10))
          .apply_backgroundsegmenter(bgs, learningRate=-1, roi=(10, 570, 10, 1250))
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test2_MOG2.mp4', framerate=vfs.avgframerate))

### Create a coordinate array for storage of the results

We wrote the movement detection results (suprathreshold pixels) above to a video so that we could view the results. However, if you want to save the results for further analyses, it is much better to detect non-zero pixel (i.e. foreground) coordinates, and save that to a `CoordinateArrays`. We also save some metadata.

In [ ]:
%%time
coordsarray = (vfs.iter_frames(color=False)
               .apply_backgroundsegmenter(bgs, learningRate=-1)
               .morphologyex(morphtype='open', kernelsize=2)
               .save_nonzero(filepath='output/testcoordsMOG.darr',
                             metadata={'bgsparams': bgs.get_params(),
                                       'morphologyex': ('open', 2),
                                       'learningrate': -1,
                                       'avgframerate': vfs.avgframerate},
                             ignore_firstnframes=10,
                             overwrite=True))

coordsarray

For more information on `CoordinateArrays`, and how to access and view the them, just take a look at the next notebook.

# A different algorithm: KNN

In [ ]:
bgs = bw.BackgroundSubtractorKNN()
bgs.get_params()

A different background algorithm with different parameters. You can change these parameters the same way as in the example above.

In [ ]:
bgs = bw.BackgroundSubtractorKNN(kNNSamples=0)

And run the whole pipeline again with the other background subtractor, and save the results as video.

In [ ]:
frames = (vfs.iter_frames(color=False)
          .apply_backgroundsegmenter(bgs, learningRate=-1)
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test_KNN.mp4', framerate=vfs.avgframerate))

Look at results:

In [ ]:
frames.show()

# Yet another algorithm: LSBP

In [ ]:
bgs = bw.BackgroundSubtractorLSBP()
bgs.get_params()

In [ ]:
bgs = bw.BackgroundSubtractorLSBP(nSamples=10)

In [ ]:
frames = (vfs.iter_frames(color=False)
          .apply_backgroundsegmenter(bgs, learningRate=-1)
          .morphologyex(morphtype='open', kernelsize=2)
          .tovideo('output/test_LSBP.mp4', framerate=vfs.avgframerate))

In [ ]:
frames.show()